In [1]:
!pip install face_recognition opencv-python

  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached click-8.2.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl.metadata (9.2 kB)
Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)
Using cached click-8.2.1-py3-none-any.whl (102 kB)
Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl (12.7 MB)
Using cached pillow-11.3.0-cp312-cp312-win_amd64.whl (7.0 MB)

   ---------------------------------------- 0/6 [face-recognition-models]
   ---------------------------------------- 0/6 [face-recognition-models]
   ---------------------------------------- 0/6 [face-recognition-models]
   ------ ------------------------------

In [ ]:
import face_recognition
import cv2
import os

# Load known faces
known_face_encodings = []
known_face_names = []

for file in os.listdir("known_faces"):
    if file.endswith(".jpg") or file.endswith(".png"):
        image = face_recognition.load_image_file(f"known_faces/{file}")
        encodings = face_recognition.face_encodings(image)
        if encodings:  # Check if encoding was successful
            known_face_encodings.append(encodings[0])
            known_face_names.append(os.path.splitext(file)[0])

# Start webcam
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_frame)
    try:
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
    except Exception as e:
        print("Encoding error:", e)
        continue

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            index = matches.index(True)
            name = known_face_names[index]

        # Draw box and label
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
cv2.destroyAllWindows()


In [5]:
import face_recognition
import cv2
import pickle

# Load known face encodings
ENCODINGS_FILE = "encodings.pkl"

try:
    with open(ENCODINGS_FILE, "rb") as f:
        data = pickle.load(f)
        known_face_encodings = data["encodings"]
        known_face_names = data["names"]
        print(f"✅ Loaded {len(known_face_encodings)} known faces.")
except FileNotFoundError:
    print(f"❌ Error: Encoding file '{ENCODINGS_FILE}' not found.")
    exit()

# Initialize webcam
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("❌ Unable to access the webcam.")
    exit()

print("🎥 Starting face recognition. Press 'q' to quit.")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("❌ Failed to read from webcam.")
        break

    # Convert the frame from BGR to RGB
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    # Match detected faces with known faces
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        # Use the first match found (can be improved with distance calculation)
        if True in matches:
            match_index = matches.index(True)
            name = known_face_names[match_index]

        # Draw bounding box and name
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), (0, 255, 0), cv2.FILLED)
        cv2.putText(frame, name, (left + 5, bottom - 7),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Display the frame
    cv2.imshow("Live Face Recognition", frame)

    # Press 'q' to quit
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

# Clean up
video_capture.release()
cv2.destroyAllWindows()
print("🛑 Face recognition stopped.")


✅ Loaded 1 known faces.
🎥 Starting face recognition. Press 'q' to quit.
🛑 Face recognition stopped.


In [7]:
import face_recognition
import cv2
import pickle
import os
import time
from collections import deque

# --- Config ---
ENCODINGS_FILE = "encodings.pkl"
VIDEO_OUTPUT_DIR = "theft_footage"
CONFIDENCE_THRESHOLD = 0.5  # Lower = stricter
RECORD_DURATION = 10        # Seconds after detection
FPS_FALLBACK = 24           # If webcam doesn't report FPS

os.makedirs(VIDEO_OUTPUT_DIR, exist_ok=True)

# Load shoplifter encodings only
try:
    with open(ENCODINGS_FILE, "rb") as f:
        data = pickle.load(f)
        known_face_encodings = data["encodings"]
        known_face_names = data["names"]
        print(f"✅ Loaded {len(known_face_encodings)} shoplifters.")
except FileNotFoundError:
    print(f"❌ Encoding file '{ENCODINGS_FILE}' not found.")
    exit()

# Webcam init
video_capture = cv2.VideoCapture(0)
if not video_capture.isOpened():
    print("❌ Unable to access the webcam.")
    exit()

frame_width = int(video_capture.get(3))
frame_height = int(video_capture.get(4))
fps = int(video_capture.get(cv2.CAP_PROP_FPS)) or FPS_FALLBACK

# Frame buffer for pre-event
PRE_EVENT_FRAMES = deque(maxlen=fps * RECORD_DURATION)
recording = False
event_start_time = None
video_writer = None

print("🎥 Starting live theft detection. Press 'q' to quit.")

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    detected_shoplifter = False

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        if len(distances) == 0:
            continue

        best_match_index = distances.argmin()
        confidence = 1.0 - distances[best_match_index]

        name = "Unknown"
        color = (0, 255, 0)  # Green box by default

        if distances[best_match_index] < CONFIDENCE_THRESHOLD:
            name = known_face_names[best_match_index]
            color = (0, 0, 255)  # Red box for shoplifter
            detected_shoplifter = True

        label = f"{'⚠️ ' if name != 'Unknown' else ''}{name} ({confidence:.2f})"

        cv2.rectangle(frame, (left, top), (right, bottom), color, 2)
        cv2.rectangle(frame, (left, bottom - 25), (right, bottom), color, cv2.FILLED)
        cv2.putText(frame, label, (left + 5, bottom - 7),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)

    # Store frame in buffer
    PRE_EVENT_FRAMES.append(frame.copy())

    # Start recording if shoplifter detected
    if detected_shoplifter and not recording:
        timestamp = time.strftime("%Y%m%d-%H%M%S")
        output_path = os.path.join(VIDEO_OUTPUT_DIR, f"theft_{timestamp}.avi")
        video_writer = cv2.VideoWriter(output_path,
                                       cv2.VideoWriter_fourcc(*'XVID'),
                                       fps,
                                       (frame_width, frame_height))
        for buffered_frame in PRE_EVENT_FRAMES:
            video_writer.write(buffered_frame)

        recording = True
        event_start_time = time.time()
        print(f"🚨 Theft detected! Recording started: {output_path}")

    # Continue recording post-event
    if recording:
        video_writer.write(frame)

        if time.time() - event_start_time > RECORD_DURATION:
            recording = False
            event_start_time = None
            video_writer.release()
            video_writer = None
            print("💾 Theft video saved.")

    cv2.imshow("Live Theft Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

video_capture.release()
if video_writer:
    video_writer.release()
cv2.destroyAllWindows()
print("🛑 Detection stopped.")


✅ Loaded 1 shoplifters.
🎥 Starting live theft detection. Press 'q' to quit.
🚨 Theft detected! Recording started: theft_footage\theft_20250703-222639.avi
💾 Theft video saved.
🚨 Theft detected! Recording started: theft_footage\theft_20250703-222650.avi
🛑 Detection stopped.
